# Table of Contents
 <p><div class="lev1"><a href="#IPython’s-Direct-interface"><span class="toc-item-num">1&nbsp;&nbsp;</span>IPython’s Direct interface</a></div><div class="lev2"><a href="#Starting-the-IPython-controller-and-engines"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Starting the IPython controller and engines</a></div><div class="lev2"><a href="#Creating-a-DirectView-instance"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Creating a <code>DirectView</code> instance</a></div><div class="lev2"><a href="#Quick-and-easy-parallelism"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Quick and easy parallelism</a></div><div class="lev3"><a href="#Parallel-map"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Parallel map</a></div><div class="lev3"><a href="#Remote-function-decorators"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Remote function decorators</a></div><div class="lev2"><a href="#Calling-Python-functions"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Calling Python functions</a></div><div class="lev3"><a href="#apply"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>apply</a></div><div class="lev3"><a href="#Blocking-execution"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Blocking execution</a></div><div class="lev3"><a href="#Non-blocking-execution"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Non-blocking execution</a></div><div class="lev3"><a href="#The-block-and-targets-keyword-arguments-and-attributes"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>The block and targets keyword arguments and attributes</a></div><div class="lev2"><a href="#Moving-Python-objects-around"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Moving Python objects around</a></div><div class="lev3"><a href="#basic-push-and-pull"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>basic push and pull</a></div><div class="lev3"><a href="#Dictionary-interface"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Dictionary interface</a></div><div class="lev3"><a href="#Scatter-and-gather"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Scatter and gather</a></div><div class="lev3"><a href="#AsyncResults-have-metadata"><span class="toc-item-num">1.5.4&nbsp;&nbsp;</span>AsyncResults have metadata</a></div><div class="lev2"><a href="#Other-things-to-look-at"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Other things to look at</a></div><div class="lev3"><a href="#How-to-do-parallel-list-comprehensions"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>How to do parallel list comprehensions</a></div><div class="lev3"><a href="#Remote-imports"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Remote imports</a></div><div class="lev3"><a href="#Parallel-exceptions"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Parallel exceptions</a></div><div class="lev2"><a href="#example:-monte-carlo-approximation-of-pi"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>example: monte carlo approximation of pi</a></div>

# IPython’s Direct interface

The direct, or multiengine, interface represents one possible way of working with a set of IPython engines. The basic idea behind the multiengine interface is that the capabilities of each engine are directly and explicitly exposed to the user. Thus, in the multiengine interface, each engine is given an id that is used to identify the engine and give it work to do. This interface is very intuitive and is designed with interactive usage in mind, and is the best place for new users of IPython to begin.

## Starting the IPython controller and engines

To follow along with this tutorial, you will need to start the IPython
controller and four IPython engines. The simplest way of doing this is to use
the `ipcluster`:

    $ ipcluster start --n=4 --profile='direct-view'

The name of the profile is of no importance. But I advise to use different names for different notebooks: this lessens the amount of possible confusion of which cluster is running the jobs for which notebook.

## Creating a ```DirectView``` instance

The first step is to import the IPython ```ipyparallel``` module and then create a Client instance:



In [1]:
from ipyparallel import Client
rc = Client(profile='direct-view')

To make sure there are engines connected to the controller, users can get a list of engine ids:

In [2]:
rc.ids

[0, 1, 2, 3]

Here we see that there are four engines ready to do work for us.

For direct execution, we will make use of a ``DirectView`` object, which can be constructed via list-access to the client:

In [3]:
dview = rc[:] # use all engines

For more information, see the in-depth explanation of Views:

http://ipyparallel.readthedocs.io/en/6.0.2/details.html#parallel-details

## Quick and easy parallelism

In many cases, you simply want to apply a Python function to a sequence of
objects, but *in parallel*. The client interface provides a simple way
of accomplishing this: using the DirectView's `DirectView.map` method.

### Parallel map


Python’s builtin ``map()`` functions allows a function to be applied to a sequence element-by-element. This type of code is typically trivial to parallelize. In fact, since IPython’s interface is all about functions anyway, you can just use the builtin ``map()`` with a ``RemoteFunction``, or a DirectView’s ``map()`` method:

In [4]:
%%time
serial_result = list(map(lambda x:x**2**2, range(30)))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 40.8 µs


In [5]:
%%time
parallel_result = dview.map_sync(lambda x:x**2**2, range(30))

CPU times: user 101 ms, sys: 20 ms, total: 121 ms
Wall time: 130 ms


In [6]:
serial_result==parallel_result

True

### Remote function decorators

Remote functions are just like normal functions, but when they are called, they execute on one or more engines, rather than locally. IPython provides two decorators:

In [7]:
@dview.remote(block=True)
def getpid():
    '''
    import library os and return the process number (pid) corresponding with
    the execution
    '''
    import os
    return os.getpid()

In [8]:
getpid()

[8951, 8950, 8949, 8965]

The ``@parallel`` decorator creates parallel functions, that break up an element-wise operations and distribute them, reconstructing the result.

In [9]:
import numpy as np
A = np.random.random((64,48))

In [10]:
@dview.parallel(block=True)
def pmul(A,B):
    return A*B

In [11]:
%%time
C_local = A*A

CPU times: user 358 µs, sys: 37 µs, total: 395 µs
Wall time: 1.06 ms


In [12]:
%%time
C_remote = pmul(A,A)

CPU times: user 19.6 ms, sys: 4.36 ms, total: 23.9 ms
Wall time: 33.3 ms


In [13]:
(C_local == C_remote).all()

True

Calling a ``@parallel`` function does not correspond to map. It is used for splitting element-wise operations that operate on a sequence or array. For ``map`` behaviour, parallel functions do have a map method.

|call                |    pfunc(seq)                 |     pfunc.map(seq)
|--------------------|-------------------------------|-------------------------------
|# of tasks          |    # of engines (1 per engine)|     # of engines (1 per engine)
|# of remote calls   |    # of engines (1 per engine)|     ``len(seq)``
|argument to remote  |    ``seq[i:j]`` (sub-sequence)|     ``seq[i]`` (single element)

A quick example to illustrate the difference in arguments for the two modes:

In [14]:
@dview.parallel(block=True)
def echo(x):
     return str(x)

In [15]:
echo(range(5))

['range(0, 2)', 'range(2, 3)', 'range(3, 4)', 'range(4, 5)']

In [16]:
echo.map(range(5))

['0', '1', '2', '3', '4']

## Calling Python functions

The most basic type of operation that can be performed on the engines is to execute Python code or call Python functions. Executing Python code can be done in blocking or non-blocking mode (non-blocking is default) using the ``View.execute()`` method, and calling functions can be done via the ``View.apply()`` method.

### apply

The main method for doing remote execution (in fact, all methods that communicate with the engines are built on top of it), is ``View.apply()``.

```
view.apply(f, *args, **kwargs)
```

available methods:

In [17]:
[x for x in dir(dview) if x.startswith('apply')]

['apply', 'apply_async', 'apply_sync']

There are various ways to call functions with IPython, and these flags are set as attributes of the View. The ``DirectView`` has just two of these flags:

**dv.block**

: bool

    whether to wait for the result, or return an AsyncResult object immediately
    
**dv.track**

: bool

    whether to instruct pyzmq to track when zeromq is done sending the message. This is primarily useful for non-copying sends of numpy arrays that you plan to edit in-place. You need to know when it becomes safe to edit the buffer without corrupting the message.
    
**dv.targets**

: int, list of ints

    which targets this view is associated with.

### Blocking execution

In blocking mode, the ``DirectView`` object (called ``dview`` in these examples) submits the command to the controller, which places the command in the engines’ queues for execution. The ``apply()`` call then blocks until the engines are done executing the command:

In [18]:
# A DirectView of all engines
# in comment, because we did this in the beginning of this notebook
#dview = rc[:]
dview.block = True
dview['a'] = 5
dview['b'] = 10
dview.apply(lambda x: a+b+x, 27)

[42, 42, 42, 42]

You can also select blocking execution on a call-by-call basis with the ``apply_sync()`` method:

In [19]:
dview.block = False
dview.apply_sync(lambda x: a+b+x, 27)

[42, 42, 42, 42]

Python commands can be executed as strings on specific engines by using a View’s ``execute`` method:

**check how the view are generated on the Client object (rc) !**

In [20]:
rc[::2].execute('c=a+b')

<AsyncResult: execute>

In [21]:
rc[1::2].execute('c=a-b')

<AsyncResult: execute>

In [22]:
dview['c'] # shorthand for dview.pull('c', block=True)

[15, -5, 15, -5]

### Non-blocking execution



In non-blocking mode, ``apply()`` submits the command to be executed and then returns a ``AsyncResult`` object immediately. The ``AsyncResult`` object gives you a way of getting a result at a later time through its ``get()`` method.

More info on the ``AsyncResult`` object:
http://ipyparallel.readthedocs.io/en/6.0.2/asyncresult.html#parallel-asyncresult

This allows you to quickly submit long running commands without blocking your local Python/IPython session:

In [23]:
# define our function
def wait(t):
    import time
    tic = time.time()
    time.sleep(t)
    return time.time()-tic

In [24]:
# In non-blocking mode
ar = dview.apply_async(wait, 3)

In [25]:
# Now block for the result
ar.get()

[3.003119707107544, 3.003082513809204, 3.003082513809204, 3.0023560523986816]

In [26]:
# Again in non-blocking mode
ar = dview.apply_async(wait, 10)

In [29]:
# Poll to see if the result is ready
ar.ready()

True

In [30]:
# ask for the result, but wait a maximum of 1 second:
ar.get(1)

[10.006775617599487,
 10.009847164154053,
 10.008778810501099,
 10.010019063949585]

**Note**

```
Note the import inside the function. This is a common model, to ensure that the appropriate modules are imported where the task is run. You can also manually import modules into the engine(s) namespace(s) via view.execute(‘import numpy’).
```

Often, it is desirable to wait until a set of ``AsyncResult`` objects are done. For this, there is a the method ``wait()``. This method takes a tuple of ``AsyncResult`` objects (or msg_ids or indices to the client’s History), and blocks until all of the associated results are ready:

In [31]:
dview.block=False
# A trivial list of AsyncResults objects
pr_list = [dview.apply_async(wait, 3) for i in range(10)]
# Wait until all of them are done
dview.wait(pr_list)
# Then, their results are ready using get() or the `.r` attribute
pr_list[0].get()

[3.0030548572540283, 3.0030221939086914, 3.001539707183838, 3.003021717071533]

### The block and targets keyword arguments and attributes

Most DirectView methods (excluding ``apply()``) accept ``block`` and ``targets`` as keyword arguments. As we have seen above, these keyword arguments control the blocking mode and which engines the command is applied to. The ``View`` class also has ``block`` and ``targets`` attributes that control the default behavior when the keyword arguments are not provided. Thus the following logic is used for ``block`` and ``targets``:

- If no keyword argument is provided, the instance attributes are used.
- The Keyword arguments, if provided overrides the instance attributes for the duration of a single call.

The following examples demonstrate how to use the instance attributes:



In [32]:
dview.targets = [0,2]
dview.block = False
ar = dview.apply(lambda : 10)
ar.get()

[10, 10]

In [33]:
dview.targets = rc.ids # all engines (4)
dview.block = True
dview.apply(lambda : 42)

[42, 42, 42, 42]

The ``block`` and ``targets`` instance attributes of the ``DirectView`` also determine the behavior of the parallel magic commands.

## Moving Python objects around

In addition to calling functions and executing code on engines, you can transfer Python objects to and from your IPython session and the engines. In IPython, these operations are called ``push()`` (sending an object to the engines) and ``pull()`` (getting an object from the engines).

### basic push and pull

In [34]:
dview.push(dict(a=1.03234,b=3453))

[None, None, None, None]

In [35]:
dview.pull('a')

[1.03234, 1.03234, 1.03234, 1.03234]

In [36]:
dview.pull('b', targets=0)

3453

In [37]:
dview.pull(('a','b'))

[[1.03234, 3453], [1.03234, 3453], [1.03234, 3453], [1.03234, 3453]]

In [38]:
dview.push(dict(c='speed'))

[None, None, None, None]

In non-blocking mode ``push()`` and ``pull()`` also return ``AsyncResult`` objects:

In [39]:
ar = dview.pull('a', block=False)
ar.get()

[1.03234, 1.03234, 1.03234, 1.03234]

### Dictionary interface


Since a Python namespace is just a ``dict``, ``DirectView`` objects provide dictionary-style access by key and methods such as ``get()`` and ``update()`` for convenience. This make the remote namespaces of the engines appear as a local dictionary. Underneath, these methods call ``apply()``:

In [40]:
dview['a']=['foo','bar']
dview['a']

[['foo', 'bar'], ['foo', 'bar'], ['foo', 'bar'], ['foo', 'bar']]

### Scatter and gather

Sometimes it is useful to partition a sequence and push the partitions to different engines. In MPI language, this is know as scatter/gather and we follow that terminology. However, it is important to remember that in IPython’s Client class, scatter() is from the interactive IPython session to the engines and gather() is from the engines back to the interactive IPython session. For scatter/gather operations between engines, MPI, pyzmq, or some other direct interconnect should be used.

In [41]:
dview.scatter('a',range(16))

In [42]:
dview['a']

[range(0, 4), range(4, 8), range(8, 12), range(12, 16)]

In [43]:
dview.gather('a')

[range(0, 4), range(4, 8), range(8, 12), range(12, 16)]

### AsyncResults have metadata

In [44]:
# In non-blocking mode
dview.block = False
ar = dview.apply_async(wait, 2)
ar.get()
ar.metadata

[{'after': [],
  'completed': datetime.datetime(2017, 4, 14, 12, 14, 29, 489954, tzinfo=tzutc()),
  'data': {},
  'engine_id': 0,
  'engine_uuid': 'b217b20b-b83c62e55cb43cd4818a0f70',
  'error': None,
  'execute_input': None,
  'execute_result': None,
  'follow': [],
  'msg_id': '385a3eb2-4e87df72f3b6543a4b129b39',
  'outputs': [],
  'received': datetime.datetime(2017, 4, 14, 12, 14, 29, 500523, tzinfo=tzutc()),
  'started': datetime.datetime(2017, 4, 14, 12, 14, 27, 486993, tzinfo=tzutc()),
  'status': 'ok',
  'stderr': '',
  'stdout': '',
  'submitted': datetime.datetime(2017, 4, 14, 12, 14, 27, 483044, tzinfo=tzutc())},
 {'after': [],
  'completed': datetime.datetime(2017, 4, 14, 12, 14, 29, 490968, tzinfo=tzutc()),
  'data': {},
  'engine_id': 1,
  'engine_uuid': 'd3ae024d-5f45fb2f78986e1ede3898c5',
  'error': None,
  'execute_input': None,
  'execute_result': None,
  'follow': [],
  'msg_id': 'a4ac765a-0c3fbfe4046b9a7d1c196169',
  'outputs': [],
  'received': datetime.datetime(201

## Other things to look at

### How to do parallel list comprehensions



In many cases list comprehensions are nicer than using the map function. While we don’t have fully parallel list comprehensions, it is simple to get the basic effect using scatter() and gather():

The %px magic executes a single Python command on the engines specified by the targets attribute of the DirectView instance

In [45]:
dview.scatter('x', range(64))
#Parallel execution on engines: [0, 1, 2, 3]
%px y = [i**10 for i in x]
y = dview.gather('y')
print(y.get()[-10:])

[210832519264920576, 253295162119140625, 303305489096114176, 362033331456891249, 430804206899405824, 511116753300641401, 604661760000000000, 713342911662882601, 839299365868340224, 984930291881790849]


### Remote imports

Sometimes you will want to import packages both in your interactive session and on your remote engines. This can be done with the ContextManager created by a DirectView’s sync_imports() method:

In [46]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


Any imports made inside the block will also be performed on the view’s engines. sync_imports also takes a local boolean flag that defaults to True, which specifies whether the local imports should also be performed. However, support for local=False has not been implemented, so only packages that can be imported locally will work this way. Note that the usual renaming of the import handle in the same line like in import matplotlib.pyplot as plt’ does not work on the remote engine, the `as plt is ignored remotely, while it executes locally. One could rename the remote handle with %px plt = pyplot though after the import.

You can also specify imports via the @require decorator. This is a decorator designed for use in Dependencies, but can be used to handle remote imports as well. Modules or module names passed to @require will be imported before the decorated function is called. If they cannot be imported, the decorated function will never execute and will fail with an UnmetDependencyError. Failures of single Engines will be collected and raise a CompositeError, as demonstrated in the next section.

In [47]:
from ipyparallel import require

In [48]:
@require('re')
def findall(pat, x):
    # re is guaranteed to be available
    return re.findall(pat, x)

In [49]:
import time
# you can also pass modules themselves, that you already have locally:
@require(time)
def wait(t):
    time.sleep(t)
    return t

**important note to remember**

```
sync_imports() does not allow import foo as bar syntax, because the assignment represented by the as bar part is not available to the import hook.
```

### Parallel exceptions

In the multiengine interface, parallel commands can raise Python exceptions, just like serial commands. But it is a little subtle, because a single parallel command can actually raise multiple exceptions (one for each engine the command was run on). To express this idea, we have a CompositeError exception class that will be raised in most cases. The CompositeError class is a special type of exception that wraps one or more other types of exceptions. Here is how it works:

In [50]:
dview.block = True
dview.execute("1/0")

CompositeError: one or more exceptions from call to method: execute
[0:execute]: ZeroDivisionError: division by zero
[1:execute]: ZeroDivisionError: division by zero
[2:execute]: ZeroDivisionError: division by zero
[3:execute]: ZeroDivisionError: division by zero

Notice how the error message printed when CompositeError is raised has information about the individual exceptions that were raised on each engine. If you want, you can even raise one of these original exceptions:

In [51]:
from ipyparallel import CompositeError

try:
    dview.execute('1/0', block=True)
except CompositeError as e:
    e.raise_exception()

RemoteError: ZeroDivisionError(division by zero)

Since you might have 100 engines, you probably don’t want to see 100 tracebacks for a simple NameError because of a typo. For this reason, CompositeError truncates the list of exceptions it will print to CompositeError.tb_limit (default is five). You can change this limit to suit your needs with:

In [52]:
from ipyparallel import CompositeError
CompositeError.tb_limit = 1
%px x=z

CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'z' is not defined
.... 3 more exceptions ...

All of this same error handling magic even works in non-blocking mode:

In [53]:
dview.block=False
ar = dview.execute('1/0')
ar.get()

CompositeError: one or more exceptions from call to method: execute
[0:execute]: ZeroDivisionError: division by zero
.... 3 more exceptions ...

## example: monte carlo approximation of pi

A simple toy problem to get a handle on multiple engines is a Monte Carlo approximation of π.

Let’s say we have a dartboard with a round target inscribed on a square board. If you threw darts randomly, and they land evenly distributed on the square board, how many darts would you expect to hit the target?

<img src="darts.png">

In [54]:
from random import random
from math import pi
dview['random'] = random

In [55]:
def mcpi(nsamples):
    s = 0
    for i in range(nsamples):
        x = random()
        y = random()
        if x*x + y*y <= 1:
            s+=1
    return 4.*s/nsamples
    
def multi_mcpi(view, nsamples):
    p = len(view.targets)
    if nsamples % p:
        # ensure even divisibility
        nsamples += p - (nsamples%p)
    
    subsamples = nsamples//p
    
    ar = view.apply(mcpi, subsamples)
    return sum(ar)/p

def check_pi(tol=1e-5, step=10, verbose=False):
    guess = 0
    spi = pi
    steps = 0
    while abs(spi-guess)/spi > tol:
        for i in range(step):
            x = random()
            y = random()
            if x*x+y*y <= 1:
                guess += 4.
        steps += step
        spi = pi*steps
        if verbose:
            print(spi, guess, abs(spi-guess)/spi)
    return steps, guess/steps

In [56]:
%%time
mcpi(int(1e7))

CPU times: user 3.64 s, sys: 0 ns, total: 3.64 s
Wall time: 3.64 s


3.141496

In [57]:
%%time
multi_mcpi(dview, int(1e7))

CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 2.17 s


3.1414816

In [58]:
check_pi()

(77960, 3.141611082606465)